In [1]:
import ghmm
import pickle
import numpy as np
from itertools import product
from pprint import pprint
import pysam
import os
import pandas
import re
import editdistance
import sys
from multiprocessing import Pool
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/alignment_lib.ipynb"

In [2]:
NMERS = 5
NSTATES = 4**NMERS

In [3]:
# args = {
#     "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_events.template.pickle",
#     "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_calling",
#     "ref": "/home/ibis/gregor.sturm/nanopore/david_eccles_bc_ideas/mouse_ref.fa",
#     "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_hmm_params_{0}mer.pickle".format(NMERS)",
#     "ncores": 4
# }

args = {
    "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_events.template.pickle",
    "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_calling",
    "ref": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_ref.fasta",
    "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_hmm_params_{0}mer.pickle".format(NMERS),
    "ncores": 62
}

In [4]:
HMM_PARAMS = pickle.load(open(args["hmm_params"], 'rb'))
HMM_PARAMS = sorted(zip(*[HMM_PARAMS["kmers"], HMM_PARAMS["means"], HMM_PARAMS["stdv"]]))

# Train Model 

In [5]:
def mk_transmat(nmers):
    """make a transition matrix assuming move=1"""
    all_kmers = [x[0] for x in HMM_PARAMS]
    n_components = len(all_kmers)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(all_kmers):
        for i, to_kmer in enumerate(all_kmers):
            p = 1/4. if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)] else 0.
            transmat[j, i] = p          
            
    return transmat.tolist()

In [6]:
def mk_transmat0(nmers):
    """make a transition matrix assuming move=0 or move=1"""
    all_kmers = [x[0] for x in HMM_PARAMS]
    n_components = len(all_kmers)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(all_kmers):
        for i, to_kmer in enumerate(all_kmers):
            p = 0
            if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (9/10.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/10.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [7]:
def mk_transmat2(nmers):
    """make a transition matrix assuming move=0 or move=1 or move=2"""
    all_kmers = [x[0] for x in HMM_PARAMS]
    n_components = len(all_kmers)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(all_kmers):
        for i, to_kmer in enumerate(all_kmers):
            p = 0
            if from_kmer[-(NMERS-2):] == to_kmer[:(NMERS-2)]:
                """move=2"""
                p = (3/10.) * (1/16.)
            elif from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (7/10.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (0/10.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [39]:
# example code for a continuous HMM with gaussian emissions


F = ghmm.Float()  # emission domain of this model

A = mk_transmat(NMERS)
B = [[x[1], x[2]] for x in HMM_PARAMS]   # parameters of emission distributions in pairs of (mu, sigma)
pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state

# generate model from parameters
model = ghmm.HMMFromMatrices(F,ghmm.GaussianDistribution(F), A, B, pi)

In [40]:
s = str(model)
print(s)

GaussianEmissionHMM(N=64)
  state 0 (initial=0.00, mu=67.75, sigma=3.49)
    Transitions: ->0 (0.25)
  state 1 (initial=0.00, mu=61.09, sigma=3.02)
    Transitions: ->1 (0.25)

  ...

  state 62 (initial=0.00, mu=52.22, sigma=2.39)
    Transitions: ->62 (0.25)
  state 63 (initial=0.00, mu=47.64, sigma=2.26)
    Transitions: ->63 (0.25)



In [10]:
def result_to_seq(result):
    states = result[0]
    all_kmers = [x[0] for x in HMM_PARAMS]
    kmers = [all_kmers[x] for x in states]
    seq = [kmer[NMERS/2] for kmer in kmers]
    return "".join(seq)

In [11]:
def predict(means):
    seq = ghmm.EmissionSequence(F, means)
    result = model.viterbi(seq)
    return result_to_seq(result)

In [12]:
predict([44.2, 44.3, 56, 58.2, 56.2, 58.1, 58.2, 60, 30.2])

u'TTTTTTTTT'

# Validate Model 

In [13]:
!pwd

/home/ibis/gregor.sturm/nanopore/own/notebooks


In [14]:
assert os.path.isfile(args["events"])

In [15]:
ref_file = args["ref"]
test = !cat {ref_file} | grep ">"
print(test)
ref = !cat {ref_file} | grep -v ">"
ref = ref[0]
print(ref[:100])

['>burn-in lambda_ref']
GGGCGGCGACCTCGCGGGTTTTCGCTATTTATGAAAATTTTCCGGTTTAAGGCGTTTCCGTTCTTCTTCGTCATAACTTAATGTTTTTATTTAAAATACC


In [16]:
file_data = pickle.load(open(args["events"], 'rb'))
file_data = [f for f in file_data if f is not None]

In [17]:
prepare_filemap(file_data)

### Make Alignment

In [18]:
fastq_file = "{0}.fastq".format(args["out_basename"])
mk_fastq(fastq_file, file_data)

In [19]:
sam_file = "{0}.sam".format(args["out_basename"])
graphmap(ref_file, fastq_file, sam_file, args["ncores"])

[Index 09:52:07] Running in fast and sensitive mode. Two indexes will be used (double memory consumption).
[Index 09:52:07] Index already exists. Loading from file.
[Index 09:52:07] Secondary index already exists. Loading from file.
[Index 09:52:08] Index loaded in 0.46 sec.
[Index 09:52:08] Memory consumption: [currentRSS = 516 MB, peakRSS = 15658 MB]

[Run 09:52:08] Automatically setting the maximum allowed number of regions: max. 500, attempt to reduce after 100
[Run 09:52:08] Reference genome is assumed to be linear.
[Run 09:52:08] Only one alignment will be reported per mapped read.
[ProcessReads 09:52:08] Reads will be loaded in batches of up to 200 MB in size.
[ProcessReads 09:52:08] Batch of 6123 reads (64 MiB) loaded in 0.49 sec. (38556840 bases)
[ProcessReads 09:52:08] Memory consumption: [currentRSS = 582 MB, peakRSS = 15658 MB]
[ProcessReads 09:52:08] Using 62 threads.
[ProcessReads 09:53:05] [CPU time: 1965.67 sec, RSS: 603 MB] Read: 6123/6123 (100.00%) [m: 5709, u: 414]  

In [20]:
bam_file = prepare_sam(args["out_basename"])

In [21]:
samfile = pysam.AlignmentFile(bam_file)
reads  = [x for x in samfile.fetch()]
len(reads)

5709

In [22]:
def basecall_read(params):
    read_name, ref_pos = params
    file_id, channel_id = get_file_and_channel(read_name)
    print(file_id, channel_id, read_name)
    sys.stdout.flush()
    
    file_obj = get_file(channel_id, file_id)
      
    events = [x["mean"] for x in file_obj["events"].to_dict("records")]
    metrichor_seq = file_obj["fastq"].split("\n")[1]
    ref_seq = "".join([ref[x] for x in ref_pos])
    called_seq = predict(events)
    
    stats = {
        "d_metrichor": int(editdistance.eval(ref_seq, metrichor_seq)),
        "d_caller": int(editdistance.eval(ref_seq, called_seq)), 
        "length": len(ref_seq),
    }

    return (read_name, called_seq, stats)

In [23]:
p = Pool(args["ncores"])
## one can not access a read in parallel (deadlock for whatever reason)
## therfore prepare input parameters outside of map
input_params = [(read.query_name, read.get_reference_positions()) for read in reads]
try:
    results = p.map(basecall_read, input_params)
except KeyboardInterrupt:
    p.terminate()

(16, 107, 'ch107_file16_read')
(22, 283, 'ch283_file22_read')
(2, 435, 'ch435_file2_read')
(18, 87, 'ch87_file18_read')
(5, 34, 'ch34_file5_read')
(5, 211, 'ch211_file5_read')
(22, 19, 'ch19_file22_read')
(9, 385, 'ch385_file9_read')
(4, 195, 'ch195_file4_read')
(22, 219, 'ch219_file22_read')
(10, 9, 'ch9_file10_read')
(14, 413, 'ch413_file14_read')
(11, 71, 'ch71_file11_read')
(4, 377, 'ch377_file4_read')
(9, 154, 'ch154_file9_read')
(43, 450, 'ch450_file43_read')
(5, 98, 'ch98_file5_read')
(1, 391, 'ch391_file1_read')
(0, 400, 'ch400_file0_read')
(25, 23, 'ch23_file25_read')
(1, 333, 'ch333_file1_read')
(16, 236, 'ch236_file16_read')
(11, 16, 'ch16_file11_read')
(6, 458, 'ch458_file6_read')
(19, 280, 'ch280_file19_read')
(20, 500, 'ch500_file20_read')
(5, 77, 'ch77_file5_read')
(11, 493, 'ch493_file11_read')
(35, 452, 'ch452_file35_read')
(21, 96, 'ch96_file21_read')
(25, 22, 'ch22_file25_read')
(20, 216, 'ch216_file20_read')
(5, 496, 'ch496_file5_read')
(2, 308, 'ch308_file2_read')


### Stats

In [32]:
headers, seqs, stats = zip(*results)

In [33]:
stats = pandas.DataFrame(list(stats))

In [34]:
stats = stats.sum(0)

In [35]:
stats

d_caller       29575648
d_metrichor    14048622
length         28996266
dtype: int64

In [36]:
print("Relative Performance: {0:5.3f}%".format(stats["d_metrichor"] * 100 /float(stats["d_caller"])))

Relative Performance: 47.501%


In [37]:
fasta_file_called = "{0}.called.fa".format(args["out_basename"])
with open(fasta_file_called, 'w') as f: 
    for header, seq in zip(headers, seqs): 
        f.write(">" + header + "\n")
        f.write(seq + "\n")

In [38]:
sam_file_called = "{0}.called.sam".format(args["out_basename"])
graphmap(ref_file, fasta_file_called, sam_file_called, args["ncores"])

[Index 12:42:30] Running in fast and sensitive mode. Two indexes will be used (double memory consumption).
[Index 12:42:30] Index already exists. Loading from file.
[Index 12:42:31] Secondary index already exists. Loading from file.
[Index 12:42:31] Index loaded in 0.44 sec.
[Index 12:42:31] Memory consumption: [currentRSS = 516 MB, peakRSS = 15804 MB]

[Run 12:42:31] Automatically setting the maximum allowed number of regions: max. 500, attempt to reduce after 100
[Run 12:42:31] Reference genome is assumed to be linear.
[Run 12:42:31] Only one alignment will be reported per mapped read.
[ProcessReads 12:42:31] Reads will be loaded in batches of up to 200 MB in size.
[ProcessReads 12:42:31] Batch of 5709 reads (34 MiB) loaded in 0.30 sec. (33080488 bases)
[ProcessReads 12:42:31] Memory consumption: [currentRSS = 551 MB, peakRSS = 15804 MB]
[ProcessReads 12:42:31] Using 62 threads.
[ProcessReads 12:42:33] [CPU time: 99.89 sec, RSS: 557 MB] Read: 5709/5709 (100.00%) [m: 0, u: 5709]      

In [31]:
prepare_sam("{0}.called".format(args["out_basename"]))

'/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_calling.called.sorted.bam'